✍ На практике источники данных редко ограничиваются одной таблицей. Например, если мы работаем с базой данных, то данные в ней могут быть представлены в виде нескольких десятков таблиц, каждая из которых несёт отдельную информацию. Если вы делаете выгрузку из базы напрямую, не объединяя таблицы в единую структуру средствами SQL, вам необходимо знать, как работать с такими таблицами средствами Pandas. 

С КАКИМИ ДАННЫМИ МЫ РАБОТАЕМ?

В этой части модуля мы будем работать с популярным датасетом MovieLens, в котором собраны логи некоторой рекомендательной системы фильмов.

Наши данные представляют собой четыре таблицы:

ratings1 и ratings2 — таблицы с данными о выставленных пользователями оценках фильмов. Они имеют одинаковую структуру и типы данных — на самом деле это две части одной таблицы с оценками фильмов.
Img

userId — уникальный идентификатор пользователя, который выставил оценку;
movieId — уникальный идентификатор фильма;
rating — рейтинг фильма.
dates — таблица с датами выставления всех оценок.
Img

date — дата и время выставления оценки фильму.


movies — таблица с информацией о фильмах.
Img

movieId — уникальный идентификатор фильма;
title — название фильма и год его выхода;
genres — жанры фильма.

Итак, представим, что нам надо получить единую таблицу, в которой будут собраны рейтинги, даты выставления рейтингов, а также информация о фильмах. Вот как мы будем действовать:

1
Склеим таблицы ratings1 и ratings2 в единую структуру.

Термин «склеить» в данном случае обозначает конкатенацию — присоединение одной таблицы к другой.
2
К полученной таблице с рейтингами подсоединим столбец с датой проставления рейтинга, склеив столбцы таблиц между собой.

3
Присоединим к нашей таблице информацию о названиях и жанрах фильмов.

ЗАЧЕМ ХРАНИТЬ ДАННЫЕ В РАЗНЫХ ТАБЛИЦАХ?

Конечно, здорово, если все необходимые данные лежат в одной таблице, но на практике такое случается редко по двум объективным причинам:

Часто данные формируются несколькими независимыми процессами, каждый из которых хранит данные в своей таблице.

Например, данные для отчёта по продажам могут состоять из списка банковских транзакций, курсов валют от Центробанка и планов отдела продаж из внутренней CRM. Все эти три таблицы, скорее всего, будут формироваться независимыми друг от друга системами. Объединять их в один отчёт придётся вам.

Хранить все данные в одной таблице часто очень накладно для ёмкости диска.

Например, названия фильмов в наших данных хранятся в отдельной небольшой таблице. А в логах, которые могут растягиваться на многие миллионы строк, вместо названия фильма стоит его идентификатор. Числовой идентификатор фильма занимает на диске гораздо меньше места, чем длинное название, поэтому логи с идентификаторами будут занимать гораздо меньше места, чем единая таблица с названиями.

In [25]:
import pandas as pd

dates = pd.read_csv('data/dates.csv')
dates.head()

,date
0,2000-07-30 18:45:03
1,2000-07-30 18:20:47
2,2000-07-30 18:37:04
3,2000-07-30 19:03:35
4,2000-07-30 18:48:51


In [26]:
movies = pd.read_csv('data/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [27]:
ratings1 = pd.read_csv('data/ratings1.csv')
ratings1.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [28]:
ratings2 = pd.read_csv('data/ratings2.csv')
ratings2.head()

,userId,movieId,rating
0,274,5621,2.0
1,274,5630,3.0
2,274,5667,3.5
3,274,5679,3.5
4,274,5690,3.0


Задание 5.1
1 point possible (graded)
Значения из какого столбца таблиц ratings1 и ratings2 можно расшифровать с помощью таблицы movies?

movieId  

In [29]:
ratings1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40001 entries, 0 to 40000
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   userId   40001 non-null  int64  
 1   movieId  40001 non-null  int64  
 2   rating   40001 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 937.6 KB


In [30]:
ratings2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60836 entries, 0 to 60835
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   userId   60836 non-null  int64  
 1   movieId  60836 non-null  int64  
 2   rating   60836 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 1.4 MB


In [31]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


Задание 5.2
1 point possible (graded)
Сколько уникальных фильмов представлено в таблице movies?

In [32]:
movies.nunique() # 9742

movieId    9742
title      9737
genres      951
dtype: int64

Задание 5.3
1 point possible (graded)
Сколько уникальных пользователей в таблице ratings1?

In [33]:
ratings1.nunique() # 274

userId      274
movieId    6219
rating       10
dtype: int64

In [34]:
dates_df.info()

NameError: name 'dates_df' is not defined

In [ ]:
dates['date'] = pd.to_datetime(dates['date'])
dates['date'].dt.year.value_counts() # 2000

date
2000    10061
2017     8198
2007     7114
2016     6703
2015     6616
2018     6418
1996     6040
2005     5813
2012     4656
2008     4351
2009     4158
2006     4059
2003     4014
2001     3922
2002     3478
2004     3279
1999     2439
2010     2301
1997     1916
2011     1690
2013     1664
2014     1439
1998      507
Name: count, dtype: int64

Следуя нашему плану объединения таблиц, первым делом мы должны склеить таблицы ratings1 и ratings2 по строкам.

Для этого воспользуемся встроенной функцией Pandas concat(), которая позволяет склеивать (конкатенировать) таблицы как по строкам, так и по столбцам.

objs — список объектов DataFrame ([df1, df2,…]), которые должны быть сконкатенированы;

axis — ось определяет направление конкатенации: 0 — конкатенация по строкам (по умолчанию), 1 — конкатенация по столбцам;

join — либо inner (пересечение), либо outer (объединение); рассмотрим этот момент немного позже;

ignore_index — по умолчанию установлено значение False, которое позволяет значениям индекса оставаться такими, какими они были в исходных данных. Если установлено значение True, параметр будет игнорировать исходные значения и повторно назначать значения индекса в последовательном порядке.

Для корректной конкатенации по строкам объединяемые таблицы должны иметь одинаковую структуру — идентичное число и имена столбцов.

Итак, давайте склеим  ratings1 и ratings2 по строкам, так как они имеют одинаковую структуру столбцов. Для этого передадим их списком в функцию concat(). Помним, что параметр axis по умолчанию равен 0, объединение происходит по строкам, поэтому не трогаем его. 

Примечание. Обратите внимание, что concat является функцией библиотеки, а не методом DataFrame. Поэтому её вызов осуществляется как pd.concat(...).

In [ ]:
ratings = pd.concat([ratings1, ratings2])
display(ratings)

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
60831,610,166534,4.0
60832,610,168248,5.0
60833,610,168250,5.0
60834,610,168252,5.0


В результате мы увеличили первую таблицу, добавив снизу строки второй таблицы.

На первый взгляд может показаться, что всё прошло успешно, однако если мы посмотрим на индексы последних строк таблицы, то увидим, что их нумерация не совпадает с количеством строк. Это может привести к некорректному объединению таблиц по ключевым столбцам на следующем этапе решения нашей задачи.

Это связано с тем, что по умолчанию concat сохраняет первоначальные индексы объединяемых таблиц, а обе наши таблицы индексировались, начиная от 0. Чтобы создать новые индексы, нужно выставить параметр ignore_index на True:

In [ ]:
ratings = pd.concat(
    [ratings1, ratings2],
    ignore_index=True
)
display(ratings)

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100832,610,166534,4.0
100833,610,168248,5.0
100834,610,168250,5.0
100835,610,168252,5.0


Казалось бы, совсем другое дело! Но это ещё не всё. Давайте узнаем количество строк в таблицах ratings и dates, ведь нам предстоит вертикально склеить их между собой:

In [ ]:
print('Число строк в таблице ratings: ', ratings.shape[0])
print('Число строк в таблице dates: ', dates.shape[0])
print(ratings.shape[0] == dates.shape[0])

# Число строк в таблице ratings: 100837
# Число строк в таблице dates: 100836
# False

Число строк в таблице ratings:  100837
Число строк в таблице dates:  100836
False


Размерность таблиц разная — как такое могло произойти?

На самом деле очень просто: при выгрузке данных информация об оценках какого-то  пользователя попала в обе таблицы (ratings1 и ratings2). В результате конкатенации случилось дублирование строк. В данном примере их легко найти — выведем последнюю строку таблицы ratings1 и первую строку таблицы ratings2:

In [ ]:
display(ratings2.tail(5))
display(ratings1.tail(5))

,userId,movieId,rating
60831,610,166534,4.0
60832,610,168248,5.0
60833,610,168250,5.0
60834,610,168252,5.0
60835,610,170875,3.0


,userId,movieId,rating
39996,274,5582,2.5
39997,274,5585,3.5
39998,274,5588,3.5
39999,274,5618,4.0
40000,274,5621,2.0


Чтобы очистить таблицу от дублей, мы можем воспользоваться методом DataFrame drop_duplicates(), который удаляет повторяющиеся строки в таблице. Не забываем обновить индексы после удаления дублей, выставив параметр ignore_index в методе drop_duplicates() на значение True:

In [ ]:
ratings = ratings.drop_duplicates(ignore_index=True)
print('Число строк в таблице ratings: ', ratings.shape[0])
# Число строк в таблице ratings: 100836

Число строк в таблице ratings:  100836


Наконец, мы можем добавить к нашей таблице с оценками даты их выставления. Для этого конкатенируем таблицы ratings и dates по столбцам:

In [ ]:
ratings_dates = pd.concat([ratings, dates], axis=1)
display(ratings_dates.tail(3))

,userId,movieId,rating,date
100833,610,168250,5.0,2017-05-08 19:50:47
100834,610,168252,5.0,2017-05-03 21:19:12
100835,610,170875,3.0,2017-05-03 21:20:15


✍ Итак, мы смогли создать единую таблицу с рейтингами и датами их представления. 

ТИПЫ ОБЪЕДИНЕНИЙ

Типы объединений в Pandas тесно связаны с операцией join из SQL, которую мы будем рассматривать в курсе в дальнейшем.

Они представлены на схеме ниже в виде кругов Эйлера. 